In [1]:
import os
import numpy as np
from PIL import Image

In [2]:
def load_data(folder):
    categories = ['jade', 'james', 'jane', 'joel', 'jovi']
    data = []
    labels = []
    
    for category in categories:
        path = os.path.join(folder, category)
        label = categories.index(category)
        
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            img = Image.open(img_path).convert('L')  
            img = img.resize((64, 64)) 
            data.append(np.array(img).flatten() / 255.0)  
            labels.append(label)
    
    return np.array(data), np.array(labels)

In [3]:
train_data, train_labels = load_data("train")
test_data, test_labels = load_data("test")

In [4]:
def one_hot_encode(labels, num_classes):
    return np.eye(num_classes)[labels]

num_classes = 5
train_labels = one_hot_encode(train_labels, num_classes)
test_labels = one_hot_encode(test_labels, num_classes)

In [5]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, lr=0.01):
        self.lr = lr
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01  
        self.b1 = np.zeros((1, hidden_size)) 
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01 
        self.b2 = np.zeros((1, output_size))  
    def relu(self, x):
        return np.maximum(0, x)
    
    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    
    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.relu(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.softmax(self.z2)
        return self.a2
    
    def backward(self, X, y, output):
        m = X.shape[0]
        dz2 = output - y
        dW2 = np.dot(self.a1.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        dz1 = np.dot(dz2, self.W2.T) * (self.a1 > 0)
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
        
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1
    
    def compute_loss(self, y_true, y_pred):
        m = y_true.shape[0]
        return -np.sum(y_true * np.log(y_pred + 1e-9)) / m  
    
    def train(self, X, y, epochs=100):
        for epoch in range(epochs):
            output = self.forward(X)
            loss = self.compute_loss(y, output)
            self.backward(X, y, output)
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")
    
    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)

In [6]:
input_size = 64 * 64  
hidden_size = 256
output_size = 5

In [7]:
nn = NeuralNetwork(input_size, hidden_size, output_size, lr=0.01)
nn.train(train_data, train_labels, epochs=200)

Epoch 0, Loss: 1.6059
Epoch 10, Loss: 1.5918
Epoch 20, Loss: 1.5778
Epoch 30, Loss: 1.5629
Epoch 40, Loss: 1.5467
Epoch 50, Loss: 1.5288
Epoch 60, Loss: 1.5088
Epoch 70, Loss: 1.4864
Epoch 80, Loss: 1.4611
Epoch 90, Loss: 1.4330
Epoch 100, Loss: 1.4021
Epoch 110, Loss: 1.3688
Epoch 120, Loss: 1.3340
Epoch 130, Loss: 1.2985
Epoch 140, Loss: 1.2630
Epoch 150, Loss: 1.2282
Epoch 160, Loss: 1.1945
Epoch 170, Loss: 1.1619
Epoch 180, Loss: 1.1306
Epoch 190, Loss: 1.1004


In [8]:
predictions = nn.predict(test_data)
true_labels = np.argmax(test_labels, axis=1)
accuracy = np.mean(predictions == true_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 67.20%
